# Web Scraping

REGALARIO, Jeremiah Daniel A.

Assigned Website: Automart.ph

Assigned Bank: PSBank

## PSBank

In [54]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [55]:
!pip install tabula
!pip install tabula-py

In [56]:
from bs4 import BeautifulSoup
import requests
import json
import numpy as np
import gc
import pandas as pd
import tabula

In [57]:
df = tabula.io.read_pdf(r"/content/drive/MyDrive/2024 BIP DIP & AGSIP/Web Scraping/pre-owned-cars-for-sale_6142024.pdf", pages='all', pandas_options={"header": None})

In [58]:
for d in df:
  d.columns = ["#", "YEAR MODEL", "MAKE/MODEL", "PLATE NO.", "COLOR", "MILEAGE (KMS)", "REGISTERED IN", "SELLING PRICE", "FLOOR PRICE"]

# Concatenate Dataframes
psbank_df = pd.concat([d for d in df])
psbank_df.reset_index(drop=True, inplace=True)

# Reset Index
psbank_df.index.name = "ID"
psbank_df.index = np.arange(0, len(psbank_df))

# Remove headers
psbank_df.drop(0, inplace=True)
psbank_df.drop("#", axis=1, inplace=True)
psbank_df.drop(8, inplace=True)

# Remove duplicate rows and null-valued rows
psbank_df.drop_duplicates(inplace=True)
psbank_df.dropna(how="any", inplace=True)

# Reset Index
psbank_df.index = np.arange(0, len(psbank_df))


# Make Transmission Column
transmission = []

for s in psbank_df["MAKE/MODEL"]:
  psbank_df.replace(s, s.replace("\r", ""), inplace=True)

for s in psbank_df["MAKE/MODEL"]:
  make = s.split("-")
  transmission.append("AUTOMATIC" if "A" in make[-1] else ("MANUAL" if "M" in make[-1] else ""))

  psbank_df.replace(s, s.replace(f"-{make[-1]}", ""), inplace=True)

# Clean Info
for s in psbank_df["COLOR"]:
  psbank_df.replace(s, s.replace("\r", " "), inplace=True)

psbank_df.insert(3, "TRANSMISSION", transmission, True)


psbank_df

,YEAR MODEL,MAKE/MODEL,PLATE NO.,TRANSMISSION,COLOR,MILEAGE (KMS),REGISTERED IN,SELLING PRICE,FLOOR PRICE
0,2023,HONDA HRV 1.5S CVT HONDA SENSING,E4-E093,AUTOMATIC,PLATINUM WHITE,"12,169",GENERAL SANTOS,"1,210,000.00","1,089,000.00"
1,2021,NISSAN NAVARA 2.5 VE CALIBRE 4X2,NEV-7786,AUTOMATIC,ALPINE WHITE,"29,444",MUNTINLUPA,"1,080,000.00","972,000.00"
2,2022,NISSAN URVAN NV350 2.5 18STR CRDI,CBD-6478,MANUAL,BRILIANT SILVER,"15,033",REGION 3,"1,050,000.00","945,000.00"
3,2019,TOYOTA FORTUNER 2.4G 4X2,CAS-9816,AUTOMATIC,ATITTUDE BLACK,"54,033",REGION 3,"1,150,000.00","1,035,000.00"
4,2022,TOYOTA FORTUNER 2.4G 4X2,NIE-1544,AUTOMATIC,SUPER WHITE II,"67,538",BALANGA,"1,560,000.00","1,404,000.00"
...,...,...,...,...,...,...,...,...,...
98,2023,TOYOTA VIOS 1.3 XLE,GAP-5099,AUTOMATIC,ALUM JADE M,"14,580",RONDA,"670,000.00","603,000.00"
99,2023,TOYOTA VIOS 1.3 XLE,Z4-Z820,AUTOMATIC,BLACKISH RED MICA,"3,932",CEBU,"650,000.00","585,000.00"
100,2023,TOYOTA VIOS 1.3 XLE,Z6-D047,AUTOMATIC,ALUMINA JADE,"11,152",CAGAYAN DE ORO,"599,000.00","542,000.00"
101,2024,TOYOTA VIOS 1.3 XLE,Z7-L191,AUTOMATIC,GRAYISH BLUE MI,"4,774",TALISAY,"694,000.00","627,000.00"


In [59]:
psbank_df.shape

(103, 9)

### Convert and Export

Convert dataframe to parquet file.

In [73]:
psbank_df["YEAR MODEL"] = psbank_df["YEAR MODEL"].astype(int)
psbank_df.to_parquet(r"/content/drive/MyDrive/2024 BIP DIP & AGSIP/Web Scraping/JD_psbank_df.parquet")

# Automart.ph

In [61]:
url = r"https://api.automart.ph/products?page=1&limit=24"
content_html = requests.get(url)
content = content_html.json()
content['items']

[{'id': 55071,
  'title': '2018 Lexus RX350 Sport  3.5',
  'slug': 'a4u697',
  'buy_now_price': 3195000,
  'price': None,
  'price_order': 3195000,
  'is_featured': 0,
  'view': 2047,
  'winner_id': None,
  'is_acuv': 0,
  'is_repoz': 0,
  'expired_at': '2024-06-28 06:20:00',
  'car': {'odometer_reading': 77000,
   'year_model': '2018',
   'model': {'name': 'RX350',
    'transmission_type': 'AT',
    'fuel_type': 'Gas',
    'vehicle_type': {'id': 3},
    'make': {'name': 'Lexus'}}},
  'acuv_certification': None,
  'warehouse': {'location': 'Baesa',
   'has_financing': 1,
   'city': {'city_name': 'Quezon City'},
   'bank': {'id': 1,
    'min_loan_amount': 260000,
    'year_minus_count': 4,
    'has_processing_fee': 0}},
  'media_images': [{'id': 373967,
    'user_id': 1,
    'ad_id': 55071,
    'post_id': None,
    'media_name': 'used-Lexus-RX350-a4u697-373967.webp',
    'url': 'https://automart-dev.sgp1.digitaloceanspaces.com/a4u697/used-Lexus-RX350-a4u697-373967.webp',
    'type': 'im

In [62]:
url_body = r"https://api.automart.ph/vehicle-types?status=1"
body_content = requests.get(url_body).json()
body_content

[{'id': 4,
  'name': 'SUV',
  'label': 'SUV',
  'image': 'https://automart-dev.sgp1.digitaloceanspaces.com/assets/vehicle_types/used-car-type-suv.webp',
  'status': 1,
  'sequence_order': 1,
  'created_at': '2019-08-02T10:56:16.000Z',
  'updated_at': '2021-12-14T23:33:03.000Z'},
 {'id': 3,
  'name': 'Sedan',
  'label': 'Sedan',
  'image': 'https://automart-dev.sgp1.digitaloceanspaces.com/assets/vehicle_types/used-car-type-sedan.webp',
  'status': 1,
  'sequence_order': 2,
  'created_at': '2019-08-02T10:56:16.000Z',
  'updated_at': '2021-12-14T23:33:03.000Z'},
 {'id': 1,
  'name': 'AUV',
  'label': 'AUV / MPV / FB',
  'image': 'https://automart-dev.sgp1.digitaloceanspaces.com/assets/vehicle_types/used-car-type-auv.webp',
  'status': 1,
  'sequence_order': 3,
  'created_at': '2019-08-02T10:56:16.000Z',
  'updated_at': '2021-12-14T23:33:03.000Z'},
 {'id': 6,
  'name': 'Pickup',
  'label': 'Pickup Trucks',
  'image': 'https://automart-dev.sgp1.digitaloceanspaces.com/assets/vehicle_types/us

In [63]:
# Get Body Types

url_body = r"https://api.automart.ph/vehicle-types?status=1"
body_type = {item["id"]: item["name"] for item in requests.get(url_body).json()}
body_type[9] = "MPV"
body_type[11] = "Truck"
body_type

{4: 'SUV',
 3: 'Sedan',
 1: 'AUV',
 6: 'Pickup',
 2: 'Hatchback',
 5: 'Van',
 9: 'MPV',
 11: 'Truck'}

In [64]:
# Count pages

url_pages = r"https://api.automart.ph/products?fields=%7B%22id%22:%22id%22%7D&limit=1&product_category_id=1"
pages = requests.get(url_pages).json()
count = pages["meta"]["totalItems"]/24
count = int(count) if count % 1 == 0 else int(count) + 1
count

47

In [65]:
def automart(url):
  content_html = requests.get(url)
  content = content_html.json()

  am_df = [{"MAKE": item["car"]["model"]["make"]["name"],
            "MODEL": item["car"]["model"]["name"],
            "YEAR": item["car"]["year_model"],
             "TITLE": item["title"].replace(item["car"]["year_model"], ""),
            "TRANSMISSION": item["car"]["model"]["transmission_type"],
            "BODY": body_type.get(item["car"]["model"]["vehicle_type"]["id"]),
            "PRICE": item["price_order"],
            "LOCATION": f"{item['warehouse']['location']}, {item['warehouse']['city']['city_name']}",
            "MILEAGE": item["car"]["odometer_reading"],
            "FUEL TYPE": item["car"]["model"]["fuel_type"] } for item in content["items"]]
  return(pd.DataFrame.from_dict(am_df))


dfs = []
for i in range(1, count + 1):
  url_am = f'https://api.automart.ph/products?page={i}&limit=24'

  df = automart(url_am).drop_duplicates().reset_index(drop=True)
  dfs.append(df)


all_df = pd.concat(dfs)

# Reset Index
all_df.index = np.arange(0, len(all_df))

all_df

,MAKE,MODEL,YEAR,TITLE,TRANSMISSION,BODY,PRICE,LOCATION,MILEAGE,FUEL TYPE
0,Lexus,RX350,2018,Lexus RX350 Sport 3.5,AT,Sedan,3195000,"Baesa, Quezon City",77000.0,Gas
1,Volvo,X40,2020,Volvo X40 T5 2.0,AT,SUV,2343000,"Baesa, Quezon City",9000.0,Gas
2,Ford,Mustang,2016,Ford Mustang V6 Coupe,AT,Sedan,2226000,"Dacudao Avenue, Davao",23509.0,Gas
3,Mercedes Benz,C250,2016,Mercedes Benz C250 Coupe,AT,Sedan,2024000,"Baesa, Quezon City",43000.0,Gas
4,Mercedes,Benz,2011,Mercedes Benz S350 3.5,AT,Van,1836450,"Kalayaan Ave, Quezon City",27000.0,Gas
...,...,...,...,...,...,...,...,...,...,...
1105,Hyundai,Accent,2020,Hyundai Accent GL 1.4,AT,Sedan,183154,"Sanford, Parañaque",NaN,Gas
1106,Nissan,Almera,2015,Nissan Almera Base 1.5,MT,Sedan,181300,"Bunawan, Davao",120897.0,Gas
1107,Hyundai,Accent,2015,Hyundai Accent E 1.4,MT,Sedan,181220,"Sanford, Parañaque",64137.0,Gas
1108,Hyundai,Eon,2016,Hyundai Eon GLS 0.8,MT,Sedan,162400,"Festival Mall, Muntinlupa",56816.0,Gas


In [66]:
all_df.shape

(1110, 10)

In [67]:
all_df[all_df["MILEAGE"].isna()]

,MAKE,MODEL,YEAR,TITLE,TRANSMISSION,BODY,PRICE,LOCATION,MILEAGE,FUEL TYPE
223,Suzuki,Ertiga,2022,Suzuki Ertiga GL 1.5,AT,MPV,641784,"Sanford, Parañaque",NaN,Diesel
276,Ford,Ranger,2019,Ford Ranger XLT 4x2 2.2,AT,Pickup,596800,"Sanford, Parañaque",NaN,Diesel
278,Ford,Ranger,2019,Ford Ranger XLT 4x2 2.2,AT,Pickup,595200,"Sanford, Parañaque",NaN,Diesel
340,Toyota,Hi-Ace,2019,Toyota Hi-Ace EL Commuter XE 2.8,MT,Van,550800,"Sanford, Parañaque",NaN,Diesel
402,Geely,EMGRAND,2023,GEELY EMGRAND EL Premium G 1.5,AT,Sedan,505096,"Bagumbayan, Taguig City",NaN,Gas
433,Ford,Everest,2017,Ford Everest Trend 4x2 2.2,AT,SUV,484653,"Oriental, Cebu",NaN,Diesel
449,Mazda,CX-3,2018,Mazda CX-3 2.0,AT,SUV,474874,"Sanford, Parañaque",NaN,Gas
515,Hyundai,HD36,2020,Hyundai HD36 2.5,MT,Van,413400,"Bagumbayan, Taguig City",NaN,Diesel
566,Toyota,Wigo,2019,Toyota Wigo G 1.0,AT,Hatchback,352000,"General Trias, Cavite",NaN,Gas
578,Honda,City,2019,Honda City E 1.5,MT,Sedan,336000,"Bunawan, Davao",NaN,Gas


In [68]:
all_df.dtypes

MAKE             object
MODEL            object
YEAR             object
TITLE            object
TRANSMISSION     object
BODY             object
PRICE             int64
LOCATION         object
MILEAGE         float64
FUEL TYPE        object
dtype: object

In [72]:
all_df.to_parquet(r"/content/drive/MyDrive/2024 BIP DIP & AGSIP/Web Scraping/JD_automart.parquet")